# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: af526a40-b3b3-4d0e-86da-7c78b160f50f
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

app_id                                           app_json  \
0  Direct Query Engine  {"app_id": "Direct Query Engine", "tags": "-",...   
1  Direct Query Engine  {"app_id": "Direct Query Engine", "tags": "-",...   
2  Direct Query Engine  {"app_id": "Direct Query Engine", "tags": "-",...   
3  Direct Query Engine  {"app_id": "Direct Query Engine", "tags": "-",...   
4  Direct Query Engine  {"app_id": "Direct Query Engine", "tags": "-",...   

                                                type  \
0  RetrieverQueryEngine(llama_index.query_engine....   
1  RetrieverQueryEngine(llama_index.query_engine....   
2  RetrieverQueryEngine(llama_index.query_engine....   
3  RetrieverQueryEngine(llama_index.query_engine....   
4  RetrieverQueryEngine(llama_index.query_engine....   

                                      record_id  \
0  record_hash_1ec4baf64c863433765412313b835579   
1  record_hash_3e7aabb789f6b1f580f1d9325ca68f90   
2  record_hash_d6e09d32cb31a9c791a7ebe722515019   
3  record_hash_c39e1a20d3b079acaad97e399218062a   
4  record_hash_1f44195eb579f8b7fa7f8b1c8a15c42f   

                                               input  \
0    "What are the keys to building a career in AI?"   
1    "How can teamwork contribute to success in AI?"   
2      "What is the importance of networking in AI?"   
3  "What are some good habits to develop for a su...   
4  "How can altruism be beneficial in building a ...   

                                              output tags  \
0  "Learning foundational technical skills, worki...    -   
1  "Teamwork can contribute to success in AI by a...    -   
2  "Networking is crucial in AI as it helps indiv...    -   
3  "Developing good habits in areas such as eatin...    -   
4  "Helping others during your career journey can...    -   

                                         record_json  \
0  {"record_id": "record_hash_1ec4baf64c863433765...   
1  {"record_id": "record_hash_3e7aabb789f6b1f580f...   
2  {"record_id": "record_hash_d6e09d32cb31a9c791a...   
3  {"record_id": "record_hash_c39e1a20d3b079acaad...   
4  {"record_id": "record_hash_1f44195eb579f8b7fa7...   

                                           cost_json  \
0  {"n_requests": 1, "n_successful_requests": 1, ...   
1  {"n_requests": 1, "n_successful_requests": 1, ...   
2  {"n_requests": 1, "n_successful_requests": 1, ...   
3  {"n_requests": 1, "n_successful_requests": 1, ...   
4  {"n_requests": 1, "n_successful_requests": 1, ...   

                                           perf_json  \
0  {"start_time": "2025-01-13T17:34:13.997329", "...   
1  {"start_time": "2025-01-13T17:34:15.177496", "...   
2  {"start_time": "2025-01-13T17:34:16.372393", "...   
3  {"start_time": "2025-01-13T17:34:17.713445", "...   
4  {"start_time": "2025-01-13T17:34:18.848883", "...   

                           ts  Answer Relevance  Context Relevance  \
0  2025-01-13T17:34:15.053128               1.0               1.00   
1  2025-01-13T17:34:16.258690               1.0               0.50   
2  2025-01-13T17:34:17.600182               1.0               0.45   
3  2025-01-13T17:34:18.731040               1.0               0.90   
4  2025-01-13T17:34:19.470751               1.0                NaN   

   Groundedness                             Answer Relevance_calls  \
0           1.0  [{'args': {'prompt': 'What are the keys to bui...   
1           1.0  [{'args': {'prompt': 'How can teamwork contrib...   
2           NaN  [{'args': {'prompt': 'What is the importance o...   
3           NaN  [{'args': {'prompt': 'What are some good habit...   
4           NaN  [{'args': {'prompt': 'How can altruism be bene...   

                             Context Relevance_calls  \
0  [{'args': {'prompt': 'What are the keys to bui...   
1  [{'args': {'prompt': 'How can teamwork contrib...   
2  [{'args': {'prompt': 'What is the importance o...   
3  [{'args': {'prompt': 'What are some good habit...   
4                                                NaN   

               

In [17]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [18]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [19]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [20]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [21]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, you can begin by identifying a project that aligns with your career goals and interests. It's important to choose a project that is responsible, ethical, and beneficial to people. Once you have selected a project, you can follow the steps outlined in the chapters provided, such as scoping the project, executing it with an eye toward career development, and building a portfolio that demonstrates skill progression. This approach will help you embark on a meaningful AI project that contributes positively to your career growth.


In [22]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [23]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and perspectives of their colleagues. Working collaboratively enables team members to combine their technical insights, make informed decisions on technical matters such as new architectures or data collection strategies, and ultimately improve the overall quality of the project. Additionally, being surrounded by motivated and hardworking colleagues can inspire individuals to strive for continuous learning and development, leading to better outcomes in AI projects.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field

In [24]:
tru.get_leaderboard(app_ids=[])

Answer Relevance  Context Relevance  \
app_id                                                              
Sentence Window Query Engine               1.0               0.47   

                              Groundedness   latency  total_cost  
app_id                                                            
Sentence Window Query Engine          0.56  1.272727    0.000813

In [25]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [33]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [34]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [35]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: a024a885-b054-4b50-bcf7-1d0723ed30eb.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 885e320d-9c09-49b1-ad02-dfec7b0a639f.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

To build a portfolio of AI projects, it is important to start with simple projects and progress to more complex ones over time. Communication is crucial in explaining your thought process and showcasing the value of your work. Identifying worthwhile ideas to work on is a key skill for an AI architect. By gaining experience through working on projects in various industries such as manufacturing, healthcare, climate change, agriculture, ecommerce, and advertising, you can build a diverse portfolio that demonstrates your capabilities.


In [36]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [37]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f5feb1641f0 is calling an instrumented method <function BaseQueryEngine.query at 0x7f6187053760>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f60d811e3e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f5feb1641f0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f6181e8eb90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f60d811e3e0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f60682cebf0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f6187052b00>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: fff7f794-2d0c-4494-8f95-84216e57dad3.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 1cd286fe-9cbd-4275-83d9-e42602562261.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f5feb164160 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f6186627250>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f60d811e380) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f5feb164160 is calling an instrumented method <function Refine.get_response at 0x7f6185847010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f60d811e380) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f5feb0072c0 is calling an instrumented method <function LLMPredictor.predict at 0x7f61934f1240>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are essential keys to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by others are critical for success in tackling large projects in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f5feb164160 is calling an instrumented method <function Refine.get_response at 0x7f6185847010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f60d811e380) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f5feb1641f0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f6181e8eb90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f60d811e3e0) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, difficulties in project management due to uncertainty in time estimates for achieving target accuracy, and technical challenges that researchers and professionals in the field often face.
> Merging 3 nodes into parent node.
> Parent node id: 384762dc-4a00-49ae-b766-4a26126a7655.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 104391f9-9d1d-4cb5-a81b-0858730930f8.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f5feb164160 is calling an instrumented method <function Refine.get_response at 0x7f6185847010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f60d811e380) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f5feb1641f0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f6181e8eb90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f60d811e3e0) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: ab320d02-97aa-4058-87c0-c8e76eecf77b.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: aa932a15-b64d-4140-bd23-d827eff128af.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 3c58bf86-934c-4e7f-80f0-072406ee948e.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 0164ba11-b486-4a5c-8bfb-7d6d8545adc8.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 434c5768-1b18-46c5-9aae-eea511d76d0e.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f5feb164160 is calling an instrumented method <function Refine.get_response at 0x7f6185847010>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f60d811e380) using this function.


What is the right AI job for me?
The right AI job for you may depend on whether you are looking to switch roles, industries, or both. If you are seeking your first job in AI, transitioning either roles or industries individually may be easier than attempting both simultaneously.


In [38]:
tru.get_leaderboard(app_ids=[])

Answer Relevance  Context Relevance  Groundedness  \
app_id                                                                        
Automerging Query Engine          0.990909           0.709091      0.813333   

                           latency  total_cost  
app_id                                          
Automerging Query Engine  1.818182    0.000855

In [39]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-11-8p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>